In [1]:
import os
import json
import argparse
import logging
import pickle
import random

import pandas as pd
import torch
from torch.utils import data
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger

from proteinclip import data_utils, fasta_utils, swissprot, hparams
from proteinclip import contrastive

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.chdir(os.path.join(os.getcwd(), '../'))

In [4]:
struct = pd.read_pickle('../data/structural/structural_embeddings_0_2000.pkl')
del struct

In [35]:
args = {
    "sequence_embeddings": "../data/esm2/sequence_embedding_t33_0_2000.pkl",
    'structural_embed':'../data/structural/structural_embeddings_0_2000.pkl',
    "functional_embeddings_file": "protclip_embed_dataset.parquet",
    'unitnorm': True,
    'batch_size': 64,
    'dim': 128,
    'nhidden': 1,
    'learning_rate':1e-4,
    'out':'training',
    'name':'testrun',
    'max_epochs': 1000,
    
}

In [32]:
pl.seed_everything(seed=6489)


[rank: 0] Seed set to 6489


6489

In [24]:
structural_embeddings = None
with open(args['structural_embed'], "rb") as f:
    structural_embeddings = pickle.load(f)
structural_embeddings = dict(zip(
    structural_embeddings["protein_id"],
    structural_embeddings["embedding"]
))
list(structural_embeddings.items())[0]

/tmp/ipykernel_3092007/1608996607.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  structural_embeddings = pickle.load(f)


('Q15375',
 array([ -5.27637  ,   9.696514 ,  11.407264 , ...,  -5.5494313,
        -11.50502  , -10.721539 ], shape=(1024,), dtype=float32))

In [36]:
sequence_embeddings = None
with open(args['sequence_embeddings'], "rb") as f:
    sequence_embeddings = pickle.load(f)

list(sequence_embeddings.items())[0]

('Q15375',
 array([ 0.00049137, -0.04332284, -0.01638226, ..., -0.10839974,
         0.02102051,  0.13916047], shape=(1280,), dtype=float32))

In [57]:
functional_embeddings = None
df = pd.read_parquet(args["functional_embeddings_file"])
functional_embeddings: dict[str, str] = dict(df[["id", "textual_embedding"]].values)


In [58]:
shared_keys = sorted(set(proteinclip_embeddings.keys()).intersection(set(structural_embeddings.keys())).intersection(functional_embeddings.keys()))


In [59]:
random.seed(42)  # Set your desired seed here
random.shuffle(shared_keys)


In [60]:
type(functional_embeddings)

dict

In [62]:
from proteinclip.data_utils import CLIPDataset3D
dset = data_utils.CLIPDataset3D(
    triples=shared_keys,
    map1=functional_embeddings,
    map2=sequence_embeddings,
    map3=structural_embeddings,
    enforce_unit_norm=args['unitnorm'],
)

Checking for missing/zero embeddings:   0%|          | 0/1997 [00:00<?, ?it/s]

INFO:root:Trimmed 0 triples with missing/zero embeddings; 1997 remain.


In [63]:
split_indices = data_utils.random_split(len(dset), [0.9, 0.05, 0.05])
dset_splits = [data.Subset(dset, idx) for idx in split_indices]

In [64]:
# Create data loaders
train_dl, valid_dl, _test_dl = [
    data.DataLoader(
        ds,
        batch_size=args['batch_size'],
        shuffle=(i == 0),
        # drop_last=(i == 0),
        num_workers=8,
        pin_memory=True,
    )
    for i, ds in enumerate(dset_splits)
]


/home/hice1/rmehta98/.conda/envs/compbio/lib/python3.13/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [66]:
# Define network
input_dim_1 = next(iter(train_dl))["x_1"].shape[-1]
input_dim_2 = next(iter(train_dl))["x_2"].shape[-1]
input_dim_3 = next(iter(train_dl))["x_3"].shape[-1]
model_class = (
    contrastive.ContrastiveEmbedding3D
)
net = model_class(
    input_dim_1=input_dim_1,
    input_dim_2=input_dim_2,
    input_dim_3=input_dim_3,
    shared_dim=args['dim'],
    num_hidden=args['nhidden'],
    lr=args['learning_rate'],
)

/home/hice1/rmehta98/.conda/envs/compbio/lib/python3.13/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [67]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(
    monitor='val_loss',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='min',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='min',
    filename='best-checkpoint'
)


In [69]:
# Train
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cuda",
    devices=1,
    enable_progress_bar=True,
    # logger=logger,
    # log_every_n_steps=10,
    deterministic=True,
    callbacks=[early_stop_callback, checkpoint_callback],

)
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params | Mode 
----------------------------------------------
0 | project_1    | MLP  | 1.2 M  | train
1 | project_2    | MLP  | 1.8 M  | train
2 | project_3    | MLP  | 1.2 M  | train
  | other params | n/a  | 1      | n/a  
----------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.688    Total estimated model params size (MB)
21        Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/hice1/rmehta98/.conda/envs/compbio/lib/python3.13/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 50 records. Best score: 1.911. Signaling Trainer to stop.


In [70]:



# Export model as ONNX files
contrastive.model_to_onnx(
    net.project_1,
    "project3d_1.onnx",
    input_shape=(input_dim_1,),
)
contrastive.model_to_onnx(
    net.project_2,
    "project3d_2.onnx",
    input_shape=(input_dim_2,),
)
contrastive.model_to_onnx(
    net.project_2,
    "project3d_3.onnx",
    input_shape=(input_dim_3,),
)


OnnxExporterError: Module onnx is not installed!